In [3]:
from pprint import pprint
import pymongo
from pymongo import MongoClient
import csv
from datetime import datetime
from datetime import timedelta
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import time

def samedate(hotspot_date, climate_date):
    # this function checks if two dates are the same
    # This function was made because date is forrmatted differently in hotspot_historic and climate_historic
    # eg. hotspot historic '27/12/2018' and climate historic '27/12/18'
    actual_date = []
    final_date = []
    join_string = ''
    date_length = len(hotspot_date)
    for k in range(date_length - 1, -1, -1):
        if k == 7 or k == 6:
            continue
        else:
            actual_date.append(hotspot_date[k])
    for j in range(date_length - 3, -1, -1):
        final_date.append(actual_date[j])

    return (join_string.join(final_date) == climate_date)

def get_prec_number(a_string):
    # this function gives the float part of a precipitation measurement
    final = a_string.strip()
    return float(final[0:4])
    

def get_prec_letter(a_string):
    # this function gives the letter representing the tyoe of precipitation measurement
    final = a_string.strip()
    return final[-1]

def change_format(a_date):
    date_final = datetime.strptime(a_date, '%d/%m/%y')
    formatted = date_final.strftime("20%y/%m/%d")
    return formatted

# Task 2
# Question 1
# So we have to load all the data from the csv files into a new database fit3182_assignment_db
# Check if the main collection already exists
try:
    a_collection.drop()
except NameError:
    pass
# Now create the data base
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
a_collection = db.climatee # create a new collection called 'a_collection'
with open('climate_historic.csv', 'r') as file_one:  # open climate historic file
    # now create a csv object for each of the files
    climate_data = csv.reader(file_one)
    climate_lines = []
    count = 0
    for c_line in climate_data:
        if count == 0:
            count += 1
        else:
            climate_lines.append(c_line)   # put each line of the data in a list 
            

            
with open('hotspot_historic.csv', 'r') as file_two: # open hotspot historic file
    hotspot_data = csv.reader(file_two)
    hotspot_lines = []
    count = 0
    for h_line in hotspot_data:  # go through the hotspot data.
        if count == 0:
            count += 1
        else:
            hotspot_lines.append(h_line)  # put each line of the data in a list

documents = []
for climate_line in climate_lines:
    matched = False
    for hotspot_line in hotspot_lines:
        if samedate(hotspot_line[4], climate_line[1]):
            new_entry = {
             "latitude" : float(hotspot_line[0]),
             "longitude": float(hotspot_line[1]),
             "datetime" : hotspot_line[2],
             "confidence" : float(hotspot_line[3]),
             "surface_temperature_celsius" : float(hotspot_line[5]),
            "station": float(climate_line[0]),
            "date": change_format(climate_line[1]),
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation_level": get_prec_number(climate_line[6]),
            "precipitation_type": get_prec_letter(climate_line[6]),
            "GHI_w/m2": float(climate_line[7])}
            documents.append(new_entry)
            matched = True
    if matched == False: # there was not a fire on this date
        # Set None's for all the unmatchable values cause there was no fire on this date
        no_fire = {
            "latitude" : None,
            "longitude": None,
            "datetime" : None,
            "confidence" : None,
            "surface_temperature_celsius": None,
            "station": float(climate_line[0]),
            "date": change_format(climate_line[1]),
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation_level": get_prec_number(climate_line[6]),
            "precipitation_type": get_prec_letter(climate_line[6]),
            "GHI_w/m2": float(climate_line[7])
        }
        documents.append(no_fire)


result = a_collection.insert_many(documents)  
# cursor = a_collection.find({})
# for item in cursor:
#     pprint(item)
# find the latest date in the stream

latest_date = a_collection.find().sort("date", pymongo.DESCENDING).limit(1)
for k in latest_date:
    new_date = datetime.strptime(k['date'], '%Y/%m/%d')

In [7]:
from pprint import pprint
import pymongo
from pymongo import MongoClient
import csv
from datetime import datetime
from datetime import timedelta
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import time

with open('hotspot_AQUA_streaming.csv', 'r') as file_four: # open hotspot historic file
    aqua_data = csv.reader(file_four)
    aqua_lines = []
    count = 0
    for line in aqua_data:  # go through the hotspot data.
        if count == 0:
            count += 1
        else:
            aqua_lines.append(line)  # put each line of the data in a list

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ', data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'same_topic'
    prod = 'Producer2'
    print('Publishing records..')
    producer = connect_kafka_producer()
    length_lines = len(aqua_lines)
    start_time = time.time()
    new_time = '00:00:00'
    new_time = datetime.strptime(new_time, "%H:%M:%S")
    while True:
        if time.time() - start_time <= 10:
            index_of_line = random.randrange(0, length_lines)
            new_date = new_date
            str_date = new_date.strftime('%Y/%m/%d')
            new_time = new_time + timedelta(hours=4, minutes = 48)
            final_time = new_time.strftime("%H:%M:%S")
            line_sent = aqua_lines[index_of_line] + [str_date] + [final_time] + [prod]
            data = str(line_sent)
            publish_message(producer, topic, 'parsed', data)
            sleep(2)
        else:
            index_of_line = random.randrange(0, length_lines)
            new_date = new_date + timedelta(days = 1)
            str_date = new_date.strftime('%Y/%m/%d')
            new_time = '00:00:00'
            new_time = datetime.strptime(new_time, "%H:%M:%S")
            final_time = new_time.strftime("%H:%M:%S")
            line_sent = aqua_lines[index_of_line] + [str_date] + [final_time] + [prod]
            data = str(line_sent)
            publish_message(producer, topic, 'parsed', data)
            start_time = time.time()
            sleep(2)
            

Publishing records..
Message published successfully. Data:  ['-36.9521', '143.0574', '73', '47', '2019/01/02', '04:48:00', 'Producer2']
Message published successfully. Data:  ['-36.6965', '142.7116', '88', '64', '2019/01/02', '09:36:00', 'Producer2']
Message published successfully. Data:  ['-36.275', '146.154', '86', '60', '2019/01/02', '14:24:00', 'Producer2']
Message published successfully. Data:  ['-38.0383', '144.4307', '59', '40', '2019/01/02', '19:12:00', 'Producer2']
Message published successfully. Data:  ['-36.4449', '140.9836', '91', '68', '2019/01/02', '00:00:00', 'Producer2']
Message published successfully. Data:  ['-37.443', '147.3064', '61', '41', '2019/01/03', '00:00:00', 'Producer2']
Message published successfully. Data:  ['-37.561', '148.032', '73', '34', '2019/01/03', '04:48:00', 'Producer2']
Message published successfully. Data:  ['-36.0133', '141.8094', '61', '41', '2019/01/03', '09:36:00', 'Producer2']
Message published successfully. Data:  ['-35.9701', '145.7061', 

Message published successfully. Data:  ['-36.4721', '142.2586', '88', '64', '2019/01/16', '09:36:00', 'Producer2']
Message published successfully. Data:  ['-36.6095', '142.7011', '87', '62', '2019/01/16', '14:24:00', 'Producer2']
Message published successfully. Data:  ['-36.9151', '142.0798', '77', '50', '2019/01/16', '19:12:00', 'Producer2']
Message published successfully. Data:  ['-36.3756', '143.7243', '100', '88', '2019/01/17', '00:00:00', 'Producer2']
Message published successfully. Data:  ['-36.7321', '143.1978', '54', '46', '2019/01/17', '04:48:00', 'Producer2']
Message published successfully. Data:  ['-34.5019', '141.3528', '71', '46', '2019/01/17', '09:36:00', 'Producer2']
Message published successfully. Data:  ['-37.4574', '142.9354', '59', '40', '2019/01/17', '14:24:00', 'Producer2']
Message published successfully. Data:  ['-36.9194', '143.6131', '76', '50', '2019/01/17', '19:12:00', 'Producer2']
Message published successfully. Data:  ['-37.7522', '142.3183', '66', '50', '20

KeyboardInterrupt: 

TypeError: descriptor 'strftime' requires a 'datetime.date' object but received a 'str'